In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer,SimpleImputer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.cluster import KMeans
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import LogisticRegression

In [2]:
#Train Data
train_cat=pd.read_excel(r'C:\Users\LENOVO\Downloads\widsdatathon2025\TRAIN_NEW\TRAIN_CATEGORICAL_METADATA_new.xlsx')
train_quant=pd.read_excel(r'C:\Users\LENOVO\Downloads\widsdatathon2025\TRAIN_NEW\TRAIN_QUANTITATIVE_METADATA_new.xlsx')
train_connect=pd.read_csv(r'C:\Users\LENOVO\Downloads\widsdatathon2025\TRAIN_NEW\TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv')
train_sample=pd.read_excel(r'C:\Users\LENOVO\Downloads\widsdatathon2025\SAMPLE_SUBMISSION.xlsx').set_index("participant_id")
#Test Data
test_cat=pd.read_excel(r'C:\Users\LENOVO\Downloads\widsdatathon2025\TEST\TEST_CATEGORICAL.xlsx')
test_quant=pd.read_excel(r'C:\Users\LENOVO\Downloads\widsdatathon2025\TEST\TEST_QUANTITATIVE_METADATA.xlsx')
test_connect=pd.read_csv(r'C:\Users\LENOVO\Downloads\widsdatathon2025\TEST\TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')

In [3]:
train_data=pd.concat([train_cat,train_quant,train_connect],ignore_index=True)
test_data=pd.concat([test_cat,test_quant,train_connect],ignore_index=True)

In [4]:
#Impute missing values- IMPORTANT
# Identify numerical and categorical columns
numerical_cols = train_data.select_dtypes(include=['number']).columns
categorical_cols = train_data.select_dtypes(include=['object', 'category']).columns

# Using Iterative Imputer (like regression-based imputation)
iter_imputer = SimpleImputer(strategy='median')
train_data[numerical_cols] = iter_imputer.fit_transform(train_data[numerical_cols])


In [5]:
# Identify numerical and categorical columns -IMPORTANT
numerical_cols = test_data.select_dtypes(include=['number']).columns
categorical_cols = test_data.select_dtypes(include=['object', 'category']).columns

# Using Iterative Imputer (like regression-based imputation)
iter_imputer =  SimpleImputer(strategy='median')
test_data[numerical_cols] = iter_imputer.fit_transform(test_data[numerical_cols])


In [6]:
train_copy=train_data.copy()
test_copy=test_data.copy()
train_copy.columns

Index(['participant_id', 'Basic_Demos_Enroll_Year', 'Basic_Demos_Study_Site',
       'PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race',
       'MRI_Track_Scan_Location', 'Barratt_Barratt_P1_Edu',
       'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu',
       'Barratt_Barratt_P2_Occ',
       ...
       '195throw_196thcolumn', '195throw_197thcolumn', '195throw_198thcolumn',
       '195throw_199thcolumn', '196throw_197thcolumn', '196throw_198thcolumn',
       '196throw_199thcolumn', '197throw_198thcolumn', '197throw_199thcolumn',
       '198throw_199thcolumn'],
      dtype='object', length=19928)

In [7]:
# Select only numeric columns for clustering
numeric_cols = train_copy.select_dtypes(include=['int64', 'float64']).columns

# Use those as features for clustering
sex_features = numeric_cols
adhd_features = numeric_cols

# Perform KMeans clustering
from sklearn.cluster import KMeans

kmeans_sex = KMeans(n_clusters=2, random_state=42)
sex_clusters = kmeans_sex.fit_predict(train_copy[sex_features])

train_copy['Sex_F'] = sex_clusters

kmeans_adhd = KMeans(n_clusters=2, random_state=42)
adhd_clusters = kmeans_adhd.fit_predict(train_copy[adhd_features])

train_copy['ADHD_Outcome'] = adhd_clusters


In [38]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Step 1: Prepare data
X = train_copy.drop(columns=['participant_id', 'ADHD_Outcome', 'Sex_F'])
y_adhd = train_copy['ADHD_Outcome']
y_sex = train_copy['Sex_F']
X_test = test_copy.drop(columns=['participant_id'])

# Step 2: Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# Step 3: Train-validation split
X_train, X_val, y_train_adhd, y_val_adhd = train_test_split(X_scaled, y_adhd, test_size=0.2, random_state=42)
_, _, y_train_sex, y_val_sex = train_test_split(X_scaled, y_sex, test_size=0.2, random_state=42)

# Step 4: Oversampling
ros = RandomOverSampler(random_state=42)
X_train_adhd_balanced, y_train_adhd_balanced = ros.fit_resample(X_train, y_train_adhd)
X_train_sex_balanced, y_train_sex_balanced = ros.fit_resample(X_train, y_train_sex)

# Step 5: Random Forest with higher capacity
rf_adhd = RandomForestClassifier(
    class_weight='balanced',
    n_estimators=300,
    max_depth=20,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='log2',
    random_state=42,
    n_jobs=-1
)

rf_sex = RandomForestClassifier(
    class_weight='balanced',
    n_estimators=300,
    max_depth=20,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='log2',
    random_state=42,
    n_jobs=-1
)

# Step 6: Fit
rf_adhd.fit(X_train_adhd_balanced, y_train_adhd_balanced)
rf_sex.fit(X_train_sex_balanced, y_train_sex_balanced)

# Step 7: Predict
y_pred_adhd = rf_adhd.predict(X_val)
y_pred_sex = rf_sex.predict(X_val)

# Step 8: Evaluate
print("ADHD Outcome Prediction Results:")
print("Accuracy:", accuracy_score(y_val_adhd, y_pred_adhd))
print(classification_report(y_val_adhd, y_pred_adhd))

print("Sex Prediction Results:")
print("Accuracy:", accuracy_score(y_val_sex, y_pred_sex))
print(classification_report(y_val_sex, y_pred_sex))


ADHD Outcome Prediction Results:
Accuracy: 0.9986263736263736
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       697
           1       1.00      0.97      0.98        31

    accuracy                           1.00       728
   macro avg       1.00      0.98      0.99       728
weighted avg       1.00      1.00      1.00       728

Sex Prediction Results:
Accuracy: 0.9986263736263736
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       697
           1       1.00      0.97      0.98        31

    accuracy                           1.00       728
   macro avg       1.00      0.98      0.99       728
weighted avg       1.00      1.00      1.00       728



HYPERPARAMETER TUNING 

In [47]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

# ADHD model tuning
rf_adhd_base = RandomForestClassifier(random_state=42, n_jobs=-1)

random_search_adhd = RandomizedSearchCV(
    rf_adhd_base,
    param_distributions=param_grid,
    n_iter=30,  # try 30 combinations
    scoring='accuracy',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

random_search_adhd.fit(X_train_adhd_balanced, y_train_adhd_balanced)

# Best model after search
rf_adhd_best = random_search_adhd.best_estimator_


# Sex_F model tuning
rf_sex_base = RandomForestClassifier(random_state=42, n_jobs=-1)

random_search_sex = RandomizedSearchCV(
    rf_sex_base,
    param_distributions=param_grid,
    n_iter=30,
    scoring='accuracy',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

random_search_sex.fit(X_train_sex_balanced, y_train_sex_balanced)

rf_sex_best = random_search_sex.best_estimator_


Fitting 3 folds for each of 30 candidates, totalling 90 fits


KeyboardInterrupt: 

In [39]:
rf_adhd_best

RandomForestClassifier(max_depth=20, max_features=None, n_estimators=300,
                       n_jobs=-1, random_state=42)

In [27]:
# Validation predictions
y_pred_adhd_best = rf_adhd_best.predict(X_val)
y_pred_sex_best = rf_sex_best.predict(X_val)

# Evaluation
print("Tuned ADHD Model:")
print("Accuracy:", accuracy_score(y_val_adhd, y_pred_adhd_best))
print(classification_report(y_val_adhd, y_pred_adhd_best))

print("Tuned Sex Model:")
print("Accuracy:", accuracy_score(y_val_sex, y_pred_sex_best))
print(classification_report(y_val_sex, y_pred_sex_best))


Tuned ADHD Model:
Accuracy: 0.9931318681318682
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       697
           1       0.86      1.00      0.93        31

    accuracy                           0.99       728
   macro avg       0.93      1.00      0.96       728
weighted avg       0.99      0.99      0.99       728

Tuned Sex Model:
Accuracy: 0.9931318681318682
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       697
           1       0.86      1.00      0.93        31

    accuracy                           0.99       728
   macro avg       0.93      1.00      0.96       728
weighted avg       0.99      0.99      0.99       728



FINAL OUTPUT

In [45]:
# Predict on the actual test set
adhd_test_preds = rf_adhd.predict(X_test_scaled)
sex_test_preds = rf_sex.predict(X_test_scaled)

# Combine with participant_id
final_df = test_copy[['participant_id']].copy()
final_df['ADHD_Outcome'] = adhd_test_preds
final_df['Sex_F'] = sex_test_preds

# Use iloc to limit the output to the first 304 rows
final_output = final_df.iloc[:304]
# Drop duplicates, ensuring unique participant IDs
final_output = final_output.drop_duplicates()

# Save the final output DataFrame to a CSV file
final_output.to_csv('final_predictions_cleaned3.csv', index=False)

In [157]:
train_sample['ADHD_Outcome'].value_counts(normalize=True)

ADHD_Outcome
1    0.546053
0    0.453947
Name: proportion, dtype: float64

In [29]:
final_output.shape

(304, 3)

In [41]:
final_output['ADHD_Outcome'].value_counts(normalize=True)

ADHD_Outcome
0    1.0
Name: proportion, dtype: float64

In [46]:
final_output['Sex_F'].value_counts(normalize=True)

Sex_F
0    1.0
Name: proportion, dtype: float64

In [42]:
print(X.shape)          # Shape of training features
print(X_test.shape)     # Shape of test features


(3639, 19927)
(1821, 19927)
